# Manejo de errores

### Traceback

Traceback de salida, mostrando el error "FileNotFoundError":

In [1]:
open("path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/mars.jpg'

Traceback de salida, mostrando más información acerca de la excepción:

In [2]:
def main():
    open("path/to/mars.jpg")

if __name__== "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/mars.jpg'

### Controlando las excepciones

### Try y Except de los bloques

Intentamos abrir el archivo, si este no existe, manejamos la excepción para mostrar el mensaje que el archivo no fue encontrado:

In [3]:
try:
    open("config.txt")
except FileNotFoundError:
    print("Couldn't find the config.txt file!")

Couldn't find the config.txt file!


>Se debe crear una carpeta llamada "config.txt" en el mismo directorio donde se encuentra esta kata, e intentamos leer, esto mostrará otro error:

In [4]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")

if __name__ == '__main__':
    main()

PermissionError: [Errno 13] Permission denied: 'config.txt'

Detectamos todas las excepciones posibles:

In [5]:
def main():
    try:
        configuration = open('config.txt')
    except Exception:
        print("Couldn't find the config.txt file!")

main()

Couldn't find the config.txt file!


Como vemos, el mensaje nos dice que no se encontró el archivo, sin embargo la excepción lanzada no es FileNotFoundError, sino "PermissionError", por lo cual el mensaje no refleja lo que en realidad está sucediendo.

Modificamos el código para detectar ambas excepciones por separado, de modo que al existir el directorio, se debe mostrar un mensaje diciendo que el archivo en realidad es un directorio, cuando eliminemos el directorio, el mensaje mostrado debe ser el que indica que no se encontró el archivo.

In [6]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except PermissionError:
        print("Found config.txt but it is a directory, couldn't read it")

main()

Found config.txt but it is a directory, couldn't read it


Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea `except`.

In [7]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except PermissionError:
        print("Found config.txt but it is a directory, couldn't read it")
    except (BlockingIOError, TimeoutError):
        print("Filesystem under heavy load, can't complete reading configuration file")

main()

Found config.txt but it is a directory, couldn't read it


Si necesitas acceder al error asociado a la excepción, se debe actualizar la línea `except` para incluir la palabra clave `as`.

In [8]:
try:
    open("mars.jpg")
except FileNotFoundError as err:
    print("got a problem trying to read the file:", err)

got a problem trying to read the file: [Errno 2] No such file or directory: 'mars.jpg'


Usando esta técnica, podemos acceder directamente a los atributos del error:

In [9]:
try:
    open("config.txt")
except OSError as err:
    if err.errno == 2:
        print("Couldn't find the config.txt file!")
    elif err.errno == 13:
        print("Found config.txt but couldn't read it")

Found config.txt but couldn't read it


### Generación de excepciones

Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [10]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Probemos con cinco astronautas, 100 litros de agua sobrante y dos días:

In [11]:
water_left(5, 100, 2)

'Total water left after 2 days is: -10 liters'

Esto no es muy útil, ya que una carencia en los litros sería un error. Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error:

In [12]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Ahora volvamos a ejecutarlo:

In [13]:
water_left(5, 100, 2)

RuntimeError: There is not enough water for 5 astronauts after 2 days!

En el sistema de navegación, el código para señalar la alerta ahora puede usar `RuntimeError` para generar la alerta:


In [14]:
try:
    water_left(5, 100, 2)
except RuntimeError as err:
    print("Un error ocurrió aquí:", err)

Un error ocurrió aquí: There is not enough water for 5 astronauts after 2 days!


La función `water_left()` también se puede actualizar para evitar el paso de tipos no admitidos. Intentemos pasar argumentos que no sean enteros para comprobar la salida de error:

In [15]:
water_left(5, 100, None)

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

El error de `TypeError` no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use `TypeError`, pero con un mensaje mejor:

In [16]:
def water_left(astronauts, water_left, days_left):
    for argument in [astronauts, water_left, days_left]:
        try:
            # If argument is an int, the following operation will work
            argument / 10
        except TypeError:
            # TypError will be raised only if it isn't the right type 
            # Raise the same exception but with a better error message
            raise TypeError(f"All arguments must be of type int, but received: '{argument}'")
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

Ahora volvemos a intentarlo para obtener un error mejor:

In [17]:
water_left(5, 100, None)

TypeError: All arguments must be of type int, but received: 'None'